# __Instacard Orders__
## A relational set of files describing customers' orders over time

The dataset is anonymized and contains a sample of over 3 million grocery orders from more than 200,000 Instacart users. For each user, we provide between 4 and 100 of their orders, with the sequence of products purchased in each order. We also provide the week and hour of day the order was placed, and a relative measure of time between orders.

`orders` (3.4m rows, 206k users):
* `order_id`: order identifier
* `user_id`: customer identifier
* `eval_set`: which evaluation set this order belongs in (see `SET` described below)
* `order_number`: the order sequence number for this user (1 = first, n = nth)
* `order_dow`: the day of the week the order was placed on
* `order_hour_of_day`: the hour of the day the order was placed on
* `days_since_prior`: days since the last order, capped at 30 (with NAs for `order_number` = 1)

`products` (50k rows):
* `product_id`: product identifier
* `product_name`: name of the product
* `aisle_id`: foreign key
* `department_id`: foreign key

`aisles` (134 rows):
* `aisle_id`: aisle identifier
* `aisle`: the name of the aisle

`deptartments` (21 rows):
* `department_id`: department identifier
* `department`: the name of the department

`order_products__SET` (30m+ rows):
* `order_id`: foreign key
* `product_id`: foreign key
* `add_to_cart_order`: order in which each product was added to cart
* `reordered`: 1 if this product has been ordered by this user in the past, 0 otherwise

where `SET` is one of the four following evaluation sets (`eval_set` in `orders`):
* `"prior"`: orders prior to that users most recent order (~3.2m orders)
* `"train"`: training data supplied to participants (~131k orders)
* `"test"`: test data reserved for machine learning competitions (~75k orders)

### Setup

In [113]:
import os
from glob import glob
import pandas as pd

### Read in data

In [114]:
filenames = glob('./Datasets/instacart_2017_05_01/*.csv')
print("Found the following files \n{0}".format('\n'.join(filenames)))
# dataframes = {pd.read_csv(f) for f in filenames}

instacart_dict = {}
for f in filenames:
    base = base=os.path.basename(f)
    name = os.path.splitext(base)[0]
    instacart_dict[name] = pd.read_csv(f)

Found the following files 
./Datasets/instacart_2017_05_01/products.csv
./Datasets/instacart_2017_05_01/orders.csv
./Datasets/instacart_2017_05_01/order_products__train.csv
./Datasets/instacart_2017_05_01/departments.csv
./Datasets/instacart_2017_05_01/aisles.csv
./Datasets/instacart_2017_05_01/order_products__prior.csv


Example of file's contents

In [7]:
instacart_dict["products"].head(3)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7


In [115]:
instacart_dict["order_products__train"].head(3)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0


### Merge Data into flat table (TODO)

First we need to find which keys (columns) we should index on

In [8]:
for key, value in instacart_dict.items():
    cols = sorted(instacart_dict[key].columns.values)
    print(cols)

['aisle_id', 'department_id', 'product_id', 'product_name']
['days_since_prior_order', 'eval_set', 'order_dow', 'order_hour_of_day', 'order_id', 'order_number', 'user_id']
['add_to_cart_order', 'order_id', 'product_id', 'reordered']
['department', 'department_id']
['aisle', 'aisle_id']
['add_to_cart_order', 'order_id', 'product_id', 'reordered']


### Pre-process `order_products__train`

In [116]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [117]:
# drop non-important columns
temp = instacart_dict["order_products__train"].drop(['add_to_cart_order', 'reordered'], axis=1).copy()

# print some info about product ids
unique_products = temp["product_id"].unique()
print("Count of unique product ids: {0}".format(len(unique_products)))
sorted_unique_products = sorted(unique_products)
print("Range of ids: {0} to {1}".format(sorted_unique_products[0:5], sorted_unique_products[-5:-1]))

Count of unique product ids: 39123
Range of ids: [1, 2, 3, 4, 5] to [49682, 49683, 49686, 49687]


In [118]:
temp.head(10)

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633
5,1,13176
6,1,47209
7,1,22035
8,36,39612
9,36,19660


Then need to consolidate the items into 1 transaction per row

In [119]:
grouped = temp.groupby("order_id")
df = grouped.aggregate(lambda x: list(x))

In [120]:
df.head(5)

,product_id
order_id,
1,"[49302, 11109, 10246, 49683, 43633, 13176, 472..."
36,"[39612, 19660, 49235, 43086, 46620, 34497, 486..."
38,"[11913, 18159, 4461, 21616, 23622, 32433, 2884..."
96,"[20574, 30391, 40706, 25610, 27966, 24489, 39275]"
98,"[8859, 19731, 43654, 13176, 4357, 37664, 34065..."


Then [1 hot encoded](https://pbpython.com/categorical-encoding.html) each product.

In [35]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)
df = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df.pop('product_id')),
                index=df.index,
                columns=mlb.classes_))
                      index=df.index))

In [36]:
df.head()

,1,2,3,4,5,7,8,9,10,11,...,49677,49678,49679,49680,49681,49682,49683,49686,49687,49688
order_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Get frequent itemsets of `order_products__train`

In [44]:
# if col names are numbers, need to convert them to strings (limitation of apriori implementation)
df.columns = [str(i) for i in df.columns]
frequent_itemsets = apriori(df, min_support=0.02, use_colnames=True)

In [106]:
# Make product lookup table
keys = instacart_dict["products"]["product_id"]
values = instacart_dict["products"]["product_name"]
product_dictionary = dict(zip(keys, values))

# Extract frozen itemset from apriori
itemset = [list(i) for i in frequent_itemsets["itemsets"]]

# Modify itemset in place
for i in range(len(itemset)):
    for j in range(len(itemset[i])):
        itemset[i][j] = product_dictionary[int(itemset[i][j])]

pd.concat([frequent_itemsets, pd.Series(itemset)], axis=1)

,support,itemsets,0
0,0.028672,(4605),[Yellow Onions]
1,0.030935,(4920),[Seedless Red Grapes]
2,0.023649,(5450),[Small Hass Avocado]
3,0.026713,(5876),[Organic Lemon]
4,0.022346,(8424),[Broccoli Crown]
5,0.029099,(8518),[Organic Red Onion]
6,0.117980,(13176),[Bag of Organic Bananas]
7,0.049494,(16797),[Strawberries]
8,0.022034,(19057),[Organic Large Extra Fancy Fuji Apple]
9,0.083028,(21137),[Organic Strawberries]


In [43]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(13176),(21137),0.117980,0.083028,0.023428,0.198579,2.391714,0.013633,1.144183
1,(21137),(13176),0.083028,0.117980,0.023428,0.282174,2.391714,0.013633,1.228738
